In [1]:
import face_recognition
from PIL import Image, ImageDraw
import numpy
import cv2
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Define video function
def create_video(pipeline_in, input_video, output_video):
    # Process video
    video = VideoFileClip( input_video )
    video_processed = video.fl_image( pipeline_in )
    %time video_processed.write_videofile( output_video, audio=False )
    
# Load a sample picture and learn how to recognize it.
jerry_image = face_recognition.load_image_file("known_people/Jerry Seinfeld.jpg")
jerry_face_encoding = face_recognition.face_encodings(jerry_image)[0]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True
    
# Get a reference to webcam #0
#video_capture = cv2.VideoCapture(0)    
    
# Pipeline
def pipeline( image ):
    global process_this_frame, face_locations, face_encodings, face_names
        
    # Find all facial features in all the faces in the image
#    face_landmarks_list = face_recognition.face_landmarks(image)

        # Grab a single frame of video from the camera
#    ret, frame = video_capture.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(image, (0, 0), fx=0.25, fy=0.25)

    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            match = face_recognition.compare_faces([jerry_face_encoding], face_encoding)
            name = "Unknown"
            if match[0]:
                name = "Jerry"
            face_names.append(name)
    process_this_frame = not process_this_frame

    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Draw a box around the face
        cv2.rectangle(image, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(image, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(image, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Display the resulting image
#    cv2.imshow('Video', frame)
   
    return image

# Create video
create_video(pipeline, "input.mov", "output.mp4" )

# Show video
HTML("""<video width="960" height="540" controls><source src="{0}"></video>""".format( "output.mp4" ))

# Release handle to the camera
#video_capture.release()
#cv2.destroyAllWindows()


[MoviePy] >>>> Building video output.mp4
[MoviePy] Writing video output.mp4


100%|██████████| 829/829 [00:21<00:00, 37.71it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: output.mp4 

CPU times: user 1min 6s, sys: 880 ms, total: 1min 7s
Wall time: 22.3 s
